# Deep Learning 

## Background

Machine learning has conquered our digital space in a much conceivable way. Aritifical Intelligence is the umbrella discipline that describes the varied notions of the systemic behavior akin to humans. Unlike the common held adage, **Deep Learning** isn't a part of Machine Learning per se, rather it has evolved to be an exclusive stream altogether. 

<kbd>
  <img src="./props/AI_branches.jpg">
</kbd>

It is founded on **neural networks** that have, generally, outperformed other classification algorithms like Support Vector Machines (SVM), logistic regression, etc. *Neural Networks* are a specialized set of algorithms that work on assigning and moderating weights from the original data (input), across through the layers to the classification result (ouput).  

<kbd>
  <img src="./props/deep_learning_NN_better.jpg">
</kbd>